## import libraries

In [1]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import PIL
import pywt
import numpy as np
import torch
import torch.optim
from torch.autograd import Variable

GPU = True
if GPU == True:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs",torch.cuda.device_count())
    device = 'cuda'
    if torch.cuda.device_count()==0:
        dtype = torch.FloatTensor
        device = 'cpu'
else:
    dtype = torch.FloatTensor
    device = 'cpu'
    
from scipy.fftpack import dct
from scipy.fftpack import idct
from scipy import io as sio 
import time
import scipy.io as sio
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.datasets import CIFAR10
#from torchvision.datasets import CELEBA

num GPUs 1


# load previous experiment

In [2]:
loadfile = True
if loadfile:
    plotvals_loaded = np.load('plotvals_500.npy')
    [y1,y2,y3,y4,cy1,cy2,cy3,cy4] = plotvals_loaded

## load generator pre-trained mnist network

In [ ]:
from dcgan import Generator
G1 = Generator(ngpu=1).eval()
# load weights
G1.load_state_dict(torch.load('netG_epoch_99.pth'))
if torch.cuda.is_available():
    G1 = G1.cuda()

In [ ]:
batch_size = 1
latent_size = 100
z_true = Variable(torch.randn(batch_size,latent_size,1,1).cuda(), requires_grad=False)
x_true = G1(z_true).cpu().detach().numpy().squeeze()
plt.imshow(x_true,cmap="gray")

In [ ]:
mc = 500
#generate samples z
batch_size = mc
latent_size = 100
z = Variable(torch.randn(batch_size,latent_size,1,1).cuda(), requires_grad=True)

In [ ]:
def dfdz(G,z):
    f = G(z).view(-1)
    k = z.view(-1).size()[0]
    d = f.size()[0]
    gradlist = torch.zeros([d,k],dtype=torch.float)
    for i in range(d):
        ff = f[i]
        ff.backward(retain_graph=True)
        gradlist[i,:] = z.grad.squeeze()
        z.grad = torch.zeros_like(z.grad)
    return gradlist # 784 \times 100    

In [ ]:
def dfdz2(G,z): #alternate way to compute gradients - appears to give similar values
    k = z.view(-1).size()[0]
    f = G(z).view(-1)
    
    d = f.size()[0]
    gradlist = torch.zeros([d,k],dtype=torch.float)
    for i in range(d):
        f = G(z).view(-1)
        ff = f[i]
        ff.backward()#(retain_graph=True)
        gradlist[i,:] = z.grad.squeeze()
        z.grad = torch.zeros_like(z.grad)
    return gradlist # 784 \times 100    

In [ ]:
if not loadfile:
    y1 = []
    y2 = []
    latent_size = 100
    for j in range(mc):
        z_j = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=True)
        dfmat = dfdz(G1,z_j).to('cuda')
        inp1 = torch.matmul(dfmat.T,(G1(z_j).view(-1)-G1(z_true).view(-1)))
        inp2 = torch.matmul(inp1.T,z_j.squeeze()-z_true.squeeze())
        y1.append(inp2.detach().cpu().numpy())
        normzdiff = torch.norm(z_j-z_true)**2
        y2.append(normzdiff.detach().cpu().numpy())

In [ ]:
yy = []
xx = []
beta = 800
alpha = 2
for i in range(mc):
    yy.append(y1[i])
    xx.append(alpha*y2[i]- beta) 

In [ ]:
plt.scatter(xx,yy)
plt.plot(xx,xx)
plt.legend(['t1(ztrue,z)',str(alpha)+'*t2-'+str(beta)])
plt.xlabel('t2(ztrue,z)')
plt.title('DCGAN(MNIST)')

In [ ]:
if not loadfile:
    y3 = []
    y4 = []
    latent_size = 100
    for j in range(mc):
        z_t = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=False)
        z_j = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=True)
        dfmat = dfdz(G1,z_j).to('cuda')
        inp1 = torch.matmul(dfmat.T,(G1(z_j).view(-1)-G1(z_t).view(-1)))
        inp2 = torch.matmul(inp1.T,z_j.squeeze()-z_true.squeeze())
        y3.append(inp2.detach().cpu().numpy())
        normzdiff = torch.norm(z_j-z_true)**2
        y4.append(normzdiff.detach().cpu().numpy())

In [ ]:
yy = []
xx = []
beta = 1200
alpha = 2
for i in range(mc):
    yy.append(y3[i])
    xx.append(alpha*y4[i]- beta) 

In [ ]:
plt.scatter(xx,yy)
plt.plot(xx,xx)
plt.legend(['t1(ztrue,z)',str(alpha)+'*t2-'+str(beta)])
plt.xlabel('t2(ztrue,z)')
plt.title('DCGAN(MNIST)')

# load generator pre-trained network for cifar

In [ ]:
from dcgan_cifar import Generator
G2 = Generator(ngpu=1).eval()
# load weights
G2.load_state_dict(torch.load('netG_epoch_199_cifar.pth'))
if torch.cuda.is_available():
    G2 = G2.cuda()

In [ ]:
batch_size = 1
latent_size = 100
z_true = Variable(torch.randn(batch_size,latent_size,1,1).cuda(), requires_grad=False)
x_true = G2(z_true).cpu().detach().numpy().squeeze()
plt.imshow(x_true.transpose(1, 2, 0))

In [ ]:
mc = 500
#generate samples z
batch_size = mc
latent_size = 100
z = Variable(torch.randn(batch_size,latent_size,1,1).cuda(), requires_grad=True)

In [ ]:
if not loadfile:
    cy1 = []
    cy2 = []
    latent_size = 100
    for j in range(mc):
        z_j = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=True)
        dfmat = dfdz(G2,z_j).to('cuda')
        inp1 = torch.matmul(dfmat.T,(G2(z_j).view(-1)-G2(z_true).view(-1)))
        inp2 = torch.matmul(inp1.T,z_j.squeeze()-z_true.squeeze())
        cy1.append(inp2.detach().cpu().numpy())
        normzdiff = torch.norm(z_j-z_true)**2
        cy2.append(normzdiff.detach().cpu().numpy())

In [ ]:
cyy = []
cxx = []
beta = 600
alpha = 2
for i in range(mc):
    cyy.append(cy1[i])
    cxx.append(alpha*cy2[i]- beta) 

In [ ]:
plt.scatter(cxx,cyy)
plt.plot(cxx,cxx)
plt.legend(['t1(ztrue,z)',str(alpha)+'*t2-'+str(beta)])
plt.xlabel('t2(ztrue,z)')
plt.title('DCGAN(CIFAR)')

In [ ]:
if not loadfile:
    cy3 = []
    cy4 = []
    latent_size = 100
    for j in range(mc):
        z_t = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=False)
        z_j = Variable(torch.randn(1,latent_size,1,1).cuda(), requires_grad=True)
        dfmat = dfdz(G2,z_j).to('cuda')
        inp1 = torch.matmul(dfmat.T,(G2(z_j).view(-1)-G2(z_t).view(-1)))
        inp2 = torch.matmul(inp1.T,z_j.squeeze()-z_true.squeeze())
        cy3.append(inp2.detach().cpu().numpy())
        normzdiff = torch.norm(z_j-z_true)**2
        cy4.append(normzdiff.detach().cpu().numpy())

In [ ]:
cyy = []
cxx = []
beta = 1250
alpha = 3
for i in range(mc):
    cyy.append(cy3[i])
    cxx.append(alpha*cy4[i]- beta) 

In [ ]:
plt.scatter(cxx,cyy)
plt.plot(cxx,cxx)
plt.legend(['t1(ztrue,z)',str(alpha)+'*t2-'+str(beta)])
plt.xlabel('t2(ztrue,z)')
plt.title('DCGAN(CIFAR)')

In [ ]:
if not loadfile:
    plotvals = [y1,y2,y3,y4,cy1,cy2,cy3,cy4]
    np.save('plotvals_500.npy',plotvals)